In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv("/content/drive/MyDrive/STSbenchmark/input/train.csv")
data['score'] = data['score'] / 5.0 # normalize from 0 to 1
data.head(20)

,split,genre,dataset,year,sid,score,sentence1,sentence2
0,train,main-captions,MSRvid,2012test,1,1.0000,A plane is taking off.,An air plane is taking off.
1,train,main-captions,MSRvid,2012test,4,0.7600,A man is playing a large flute.,A man is playing a flute.
2,train,main-captions,MSRvid,2012test,5,0.7600,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,train,main-captions,MSRvid,2012test,6,0.5200,Three men are playing chess.,Two men are playing chess.
4,train,main-captions,MSRvid,2012test,9,0.8500,A man is playing the cello.,A man seated is playing the cello.
5,train,main-captions,MSRvid,2012test,11,0.8500,Some men are fighting.,Two men are fighting.
6,train,main-captions,MSRvid,2012test,12,0.1000,A man is smoking.,A man is skating.
7,train,main-captions,MSRvid,2012test,13,0.3200,The man is playing the piano.,The man is playing the guitar.
8,train,main-captions,MSRvid,2012test,14,0.4400,A man is playing on a guitar and singing.,A woman is playing an acoustic guitar and sing...
9,train,main-captions,MSRvid,2012test,16,1.0000,A person is throwing a cat on to the ceiling.,A person throws a cat on the ceiling.


In [ ]:
# Split the data into training and validation sets
train_data = data.sample(frac=0.8, random_state=42)
val_data = data.drop(train_data.index)

In [ ]:
# Preprocess the training data
train_sentences1 = train_data['sentence1'].values
train_sentences2 = train_data['sentence2'].values
train_labels = train_data['score'].values

In [ ]:
# Tokenize the training sentences and convert them to sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(np.concatenate((train_sentences1, train_sentences2), axis=0))
train_sequences1 = tokenizer.texts_to_sequences(train_sentences1)
train_sequences2 = tokenizer.texts_to_sequences(train_sentences2)

In [ ]:
# Pad the training sequences to have the same length
max_sequence_length = max(max(len(seq) for seq in train_sequences1), max(len(seq) for seq in train_sequences2))
padded_sequences1 = tf.keras.preprocessing.sequence.pad_sequences(train_sequences1, maxlen=max_sequence_length)
padded_sequences2 = tf.keras.preprocessing.sequence.pad_sequences(train_sequences2, maxlen=max_sequence_length)

In [ ]:
from tensorflow.keras.layers import Reshape
from tensorflow.keras import backend as K

def hamming_distance_loss(x):
    epsilon = 1e-7
    # Compute the Hamming distance between the vectors
    normalized_x = K.l2_normalize(x[0], axis=-1)
    normalized_y = K.l2_normalize(x[1], axis=-1)

    hamming_distances = K.sum(K.abs(normalized_x - normalized_y), axis=1)
    return K.reshape(hamming_distances, (-1, 1)) + epsilon

def siamese_rnn(vocab_size, embedding_dim, max_sequence_length):
    # Shared embedding layer
    embedding_layer = Embedding(vocab_size, embedding_dim, input_length=max_sequence_length)

    # Shared LSTM layer
    lstm_layer = LSTM(128)

    # Input layers
    input_1 = Input(shape=(max_sequence_length,))
    input_2 = Input(shape=(max_sequence_length,))

    # Apply embedding and LSTM
    encoded_1 = embedding_layer(input_1)
    encoded_1 = lstm_layer(encoded_1)

    encoded_2 = embedding_layer(input_2)
    encoded_2 = lstm_layer(encoded_2)

    # Hamming distance layer
    distance = Lambda(hamming_distance_loss)([encoded_1, encoded_2])

    # Create the model
    model = Model(inputs=[input_1, input_2], outputs=distance)
    return model

In [ ]:
# Create the Siamese RNN model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
model = siamese_rnn(vocab_size, embedding_dim, max_sequence_length)

In [ ]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))

# Train the model
batch_size = 64
epochs = 10
model.fit([padded_sequences1, padded_sequences2], train_labels,
          batch_size=batch_size, epochs=epochs)

Epoch 1/10
72/72 [==============================] - 29s 346ms/step - loss: 1.7814
Epoch 2/10
72/72 [==============================] - 19s 265ms/step - loss: 0.1479
Epoch 3/10
72/72 [==============================] - 21s 294ms/step - loss: 0.0964
Epoch 4/10
72/72 [==============================] - 19s 268ms/step - loss: 0.0671
Epoch 5/10
72/72 [==============================] - 20s 279ms/step - loss: 0.0528
Epoch 6/10
72/72 [==============================] - 21s 292ms/step - loss: 0.0450
Epoch 7/10
72/72 [==============================] - 19s 266ms/step - loss: 0.0399
Epoch 8/10
72/72 [==============================] - 21s 295ms/step - loss: 0.0357
Epoch 9/10
72/72 [==============================] - 19s 269ms/step - loss: 0.0315
Epoch 10/10
72/72 [==============================] - 19s 269ms/step - loss: 0.0281


In [ ]:
def preprocess_text(text, tokenizer, max_sequence_length):
    # Apply any necessary preprocessing steps to the input text
    # For example, tokenization, padding, etc.
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_sequence_length)
    return padded_sequence


def predict_similarity(input_text_1, input_text_2, model, tokenizer, max_sequence_length, threshold):
    # Preprocess the input texts
    input_seq_1 = preprocess_text(input_text_1, tokenizer, max_sequence_length)
    input_seq_2 = preprocess_text(input_text_2, tokenizer, max_sequence_length)

    # Perform the prediction
    prediction = model.predict([input_seq_1, input_seq_2])

    # Compare the predicted similarity score to the threshold
    distance = prediction[0][0]  # Assuming prediction contains Euclidean distances
    print("Distance:", distance)
    if distance <= threshold:
        return "Similar"
    else:
        return "Dissimilar"

In [ ]:
# Preprocess the validation data
val_sentences1 = val_data['sentence1'].values
val_sentences2 = val_data['sentence2'].values
val_labels = val_data['score'].values

val_sequences1 = tokenizer.texts_to_sequences(val_sentences1)
val_sequences2 = tokenizer.texts_to_sequences(val_sentences2)

val_padded_sequences1 = tf.keras.preprocessing.sequence.pad_sequences(val_sequences1, maxlen=max_sequence_length)
val_padded_sequences2 = tf.keras.preprocessing.sequence.pad_sequences(val_sequences2, maxlen=max_sequence_length)

# Iterate over different threshold values and evaluate F1 score on the validation set
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
best_f1_score = 0.0
best_threshold = 0.0
scores = []

for threshold in thresholds:
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for i in range(len(val_data)):
        prediction = predict_similarity(val_sentences1[i], val_sentences2[i], model, tokenizer, max_sequence_length, threshold)

        if prediction == "Similar":
            prediction_label = 1
        else:
            prediction_label = 0

        if prediction_label == 1 and val_labels[i] == 1:
            true_positives += 1
        elif prediction_label == 1 and val_labels[i] == 0:
            false_positives += 1
        elif prediction_label == 0 and val_labels[i] == 1:
            false_negatives += 1

    precision = true_positives / (true_positives + false_positives + 1e-7)  # Add a small epsilon value to avoid division by zero
    recall = true_positives / (true_positives + false_negatives + 1e-7)
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-7)

    scores.append(f1_score)

    if f1_score > best_f1_score:
        best_f1_score = f1_score
        best_threshold = threshold

print("F1 Scores:", scores)
print("Best F1 Score:", best_f1_score)
print("Best Threshold:", best_threshold)

Streaming output truncated to the last 5000 lines.
Distance: 0.29961815
1/1 [==============================] - 0s 34ms/step
Distance: 0.40466303
1/1 [==============================] - 0s 37ms/step
Distance: 0.3653198
1/1 [==============================] - 0s 40ms/step
Distance: 0.11047699
1/1 [==============================] - 0s 37ms/step
Distance: 0.893546
1/1 [==============================] - 0s 36ms/step
Distance: 1e-07
1/1 [==============================] - 0s 36ms/step
Distance: 0.3376322
1/1 [==============================] - 0s 38ms/step
Distance: 0.603848
1/1 [==============================] - 0s 41ms/step
Distance: 0.9357475
1/1 [==============================] - 0s 35ms/step
Distance: 0.8929349
1/1 [==============================] - 0s 47ms/step
Distance: 1e-07
1/1 [==============================] - 0s 36ms/step
Distance: 0.43366563
1/1 [==============================] - 0s 47ms/step
Distance: 0.72814167
1/1 [==============================] - 0s 36ms/step
Distance: 0.447197